In [3]:
from tda.client import Client
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
from datetime import datetime

import seaborn as sns

import login.config as config
from login.client import TDSession

In [4]:
#1 min = 60000ms
def mins_to_ms(mins):
    return mins*60000

#function to get the period desired
def get_period(mins, from_time, df):
    start = df['datetime']> from_time
    end = df['datetime']<=(from_time+mins_to_ms(mins))
    return df.loc[start&end,['datetime','low']]

#loop through low price of period to check for sell
def did_sell(df_period, sell_price,i):
    return [row1 if row1 >= sell_price else False for row1 in df_period]

#loop through low price of period to check for sell
def did_buy(df_period, buy_price,i):
    return [row1 if row1 <= buy_price else False for row1 in df_period]
        

def wol( trig_time, profit_ratio, loss_ratio, df, i):  # win or loss?  Ps the trig_price is the open price of the next minute
    # find time to start searching original df
    i = i+1
    j = i+1
#     trig_price = df.loc[df['datetime'] == trig_time, 'open'].values[0]
    trig_price = df.iloc[i, 0]
#     df = df.loc[df['datetime'] > trig_time,:] 
    df = df.iloc[j:,:] 

    # compare prices
    for i,row in df.iterrows():
        if row[0] >= trig_price * profit_ratio:
            return 'profit'     # indicates a sell/profit
        elif row[0] <= trig_price * loss_ratio:
            return 'loss'     # indicates a sell/loss
        else:
            continue

#With the WOL function - create a comprehension that will then be used as a column to indicate win or loss

#[ wol( row[//datetime], profit_ratio, loss_ratio) for i,row in trigger_df.iterrows() ]


def wol_time( trig_time, profit_ratio, loss_ratio, df, i):  # win or loss?  Ps the trig_price is the open price of the next minute
    # find time to start searching original df
    i = i+1
    j = i+1
#     trig_price = df.loc[df['datetime'] == trig_time, 'open'].values[0]
    trig_price = df.iloc[i, 0]
#     df = df.loc[df['datetime'] > trig_time,:] 
    df = df.iloc[j:,:] 

    # compare prices
    for i,row in df.iterrows():
        if row[0] >= trig_price * profit_ratio:
            return (row[5] - trig_time)/60000     # indicates a sell/profit
        elif row[0] <= trig_price * loss_ratio:
            return (row[5] - trig_time)/60000      # indicates a sell/loss
        else:
            continue

In [10]:
# symbols = symbols[25:]
# sp500 = ["HWM", "ADM", "AES", "AFL", "AIG", "APA", "IVZ", "BAC", "TFC", "BEN", "BKR", "BK", "BSX", "BWA", "CAG", "CCL", "CNP", "COG", "TPR", "COP", "CPB", "DXC", "LUMN", "DISH", "DRE", "DVN", "EOG", "EXC", "F", "FAST", "FCX", "FE", "FITB", "FTI", "GE", "GLW", "GPS", "HAL", "HBAN", "PEAK", "HIG", "HST", "HPQ", "HRL", "INTC", "IP", "IPG", "IRM", "JNPR", "KEY", "KIM", "KR", "LKQ", "LNC", "LB", "LUV", "MET", "MO", "MOS", "MRO", "VTRS", "NI", "NOV", "NRG", "NWL", "OKE", "OXY", "PFE", "PFG", "PHM", "PPL", "PRGO", "REG", "RF", "ROL", "T", "SLB", "NLOK", "JCI", "UDR", "UNM", "USB", "VNO", "VTR", "WBA", "WDC", "WFC", "WMB", "WY", "XOM", "XRX", "ZION", "TAP", "HFC", "CSCO", "MGM", "DISCA", "CF", "UAA", "VIAC"]
# symbols = sp500
symbols = ["FTI", "NYCB", "CDE", "BPFH", "MAC", "PBR.A", "QRTEA", "FCEL", "M", "PBR", "STKL", "CNX", "PVG", "ERIC", "UNIT", "ANGI", "CNHI", "HBAN", "FHN", "PRTS", "CS", "TTM", "MUR", "CLDR", "MIK", "CCJ", "MTDR", "AG", "HST", "EGO", "INSG", "UBS", "NOV", "WES", "IBN", "SOL", "PE", "CLF", "HRB", "RF", "PRMW", "KEY", "OVV", "SBSW", "BPY", "VOD", "VALE", "RIOT", "EQNR", "DVN", "KPTI", "INFY", "GME", "SPWH", "NGLOY", "MFC", "VTRS", "STL", "FCAU", "NWSA", "CLR", "SWBI", "TECK", "VST", "NLSN", "AEO", "EXEL", "BHC", "CNP", "SSRM", "FEYE", "NWL", "WKHS", "ANF", "UNM", "ATOM", "NI", "BLDP", "AA", "AES", "LBTYK", "MOS", "MT", "AU", "LBTYA", "JEF", "CNQ", "HFC", "CLSK", "DISCK", "DXC", "HWM", "PPL", "PACB", "FCX", "VIPS", "MGNI", "ABB", "T", "TRIP"]
# symbols = ["NEOS", "DDD", "MEIP", "CAKE", "ALSK", "WORX", "BRQS", "ERJ", "SNAP", "NKTR", "BRFS", "GPRO", "WEN", "BIG", "ALT", "ENLC", "UAL", "HA", "BXMT", "WTI", "OHI", "IGC", "FMCC", "NRZ", "PHM", "FUV", "CIM", "GPS", "KWBT", "INO", "VSYM", "JCPNQ", "PMT", "AAL", "RMHB", "OLN", "ADVM", "FLS", "ADMA", "VGTL", "SKT", "BBBY", "SPCE", "SWN", "AQB", "LIFE", "UE", "AGEN", "CLNY", "INFN", "WMC", "ENBL", "HMHC", "REI", "URBN", "TLGT", "CXW", "KNDI", "SRNE", "CHMA", "MGTI", "OVID", "TOL", "BB", "CERS", "CDEV", "ET",  "BOX", "IRWD", "SPWR", "GORO", "TMHC", "KDMN", "CLVS", "NGL", "RBNW",  "CBLAQ", "PEIX"]
# symbols = ['AAPL']
# symbols = symbols[4:]

# # use today: # 
# symbols = ['UAVS', 'RIOT', 'ELYS', 'MVIS', 'OPTT', 'MYOV', 'NNDM', 'SOL', 'NK', 'MGNI', 'VUZI', 'SUNW', 'CLNE', 'NCTY', 'IMMR', 'ETM', 'MBIO', 'WBAI', 'ONCT', 'CLSD', 'GME', 'UUUU', 'PACB', 'AMRS']

In [11]:
days = 10
c = pd.DataFrame({})
sym_dict = []
real_roi = list()


for i,symbol in enumerate(symbols):
    
    
    
# Get the price history iteratively
    data = TDSession.get_price_history(
        symbol, 
        period_type=Client.PriceHistory.PeriodType.DAY,
        period=Client.PriceHistory.Period.TEN_DAYS,
        frequency_type=Client.PriceHistory.FrequencyType.MINUTE,
        frequency=Client.PriceHistory.Frequency.EVERY_MINUTE)
    data = data.json()

    
    
    
# Create dataframe with new features
    test = pd.DataFrame(data['candles'])
    test['time'] = test.apply(lambda row: datetime.fromtimestamp(row.datetime/1000).strftime("%I:%M:%S"),axis=1 )
    test['day'] = test.apply(lambda row: datetime.fromtimestamp(row.datetime/1000).strftime("%d"),axis=1 )
    test['month'] = test.apply(lambda row: datetime.fromtimestamp(row.datetime/1000).strftime("%B"),axis=1 )
    test['year'] = test.apply(lambda row: datetime.fromtimestamp(row.datetime/1000).strftime("%Y"),axis=1 )
    #create open to high column
    test['open_high'] = test['high']-test['open']
    #create open to low column
    test['open_low'] = test['open']-test['low']
    
    # add EMA for previous 12 days
    test['ema_12'] = test.open.ewm(span=12).mean()
    
    # add EMA for previous 26 days
    test['ema_26'] = test.open.ewm(span=26).mean()
    
    # subtract the 12 day ema from the 26 ema
    test['up_trend'] = test.ema_12 - test.ema_26
    
    # add SMA 
    test['sma_60'] = test.open.rolling(60).mean()
    test['sma_360'] = test.open.rolling(240).mean()
    
    # subtract the 60 min ema from the 360 min
    test['up_trend_simple'] = test.sma_60 - test.sma_360
    
    test['sma_buy'] = [ True if val[17] >= 0 else False for i,val in test.iterrows() ]
    
    
    
    
    
# Define the conditions for the trigger
    #define variables
    candle_diff = 1.2
    morn15 = ['08:{}:00'.format(30+x) for x in range(0,15)]
    
    # recent low mean
    recent_low_mean = test.iloc[-600:,2].mean()

    # past high mean
    past_high_mean = test.iloc[0:600,1].mean()

    #define open higher than open low
    high_not_zero = test['open_high'] != 0 
    low_not_zero = test['open_low'] != 0 
    tall_candle = test['open_high'] > candle_diff*test['open_low']
    volume75 = test['volume'] >= np.percentile(test['volume'],[40])[0] #90
    not_first15 = ~test['time'].isin(morn15)
    price65th = test['open']<np.percentile(test['high'],[65])[0] #65
#     down_trend = past_high_mean < recent_low_mean * 1.02
#     price35th = test['open']>np.percentile(test['high'],[45])[0]

    condition = high_not_zero & low_not_zero & tall_candle & volume75 & not_first15 & price65th#  & price35th & down_trend

    
    
    
# Checking for trigger, store in trigger_df
    trigger_df = test.loc[condition, :].reset_index()

    
    
    
    
#                                       _______________-    ROI    -______________________


#add column of desired sell prices and loss prices
    #desired roi is 0.2%
    roi = 0.01
    
    my_profit = 1+roi
    trigger_df['to_sell_price'] = trigger_df.open * my_profit
    
    my_loss = 1-(roi/2)
    trigger_df['to_loss_price'] = trigger_df.open * my_loss
    
    trigger_df['wol'] = [ wol( row[6], my_profit, my_loss, test, row[0]) for i,row in trigger_df.iterrows() ]
    
    trigger_df['wol_time'] = [ wol_time( row[6], my_profit, my_loss, test, row[0]) for i,row in trigger_df.iterrows() ]
    

    
    
    
    
    
    
    
    
    
# Define time period after trigger to analyze
    sell_dur_min = 15
    
    
## Bought
    #the time period that you want to get after the trigger
    time_period6 = [get_period(6, row[6], test) for i,row in trigger_df.iterrows()]

    #did we buy???
    trigger_df['bought'] = pd.DataFrame([did_buy(time_period6[i].low,row[1],i) for i,row in trigger_df.iterrows()]).any(axis=1)

    
    
    
    
## Sold
    #the time period that you want to get after the trigger
    time_period20 = [get_period(sell_dur_min, (row[6]), test) for i,row in trigger_df.iterrows()]

    #did we sell???
    sold = pd.DataFrame([did_sell(time_period20[i].low,row[13],time_period20[i].datetime) for i,row in trigger_df.iterrows()])
    #sold_time = [did_sell(time_period20[i].low,row[13],time_period20[i].datetime) for i,row in trigger_df.iterrows()][0][1]
    trigger_df['sold'] = pd.DataFrame(sold).any(axis=1)


    ## Lost
    #the time period that you want to get after the trigger
    time_period20 = [get_period(sell_dur_min, (row[6]+60000), test) for i,row in trigger_df.iterrows()]

    #did we lose???
    trigger_df['lost'] = pd.DataFrame([did_buy(time_period20[i].low,(row[1]*.10),i) for i,row in trigger_df.iterrows()]).any(axis=1)

    trigger_df['lost'].sum()
    trigger_df.loc[trigger_df['lost']==False,:]

    trigger_df.day.value_counts().sort_index()

    profit = (trigger_df['bought'] == True) & (trigger_df['sold'] == True)
    len(trigger_df.loc[profit,:])

    loss = (trigger_df['bought'] == True) & (trigger_df['sold'] == False)
    
    no_buy = (trigger_df['bought'] == False)
#     print(trigger_df.loc[loss,:])

    min_taken = [(test.loc[(test['high']>=x[13])&(test['datetime']>x[6]), ['datetime']].iloc[0]-x[6])/60000 if test.loc[(test['high']>=x[13])&(test['datetime']>x[6]), 'datetime'].any() == True else 9999999 for i,x in trigger_df.loc[profit|loss|no_buy,:].iterrows()]
#     min_df = trigger_df.drop(trigger_df.loc[trigger_df['bought']==False,:].index) 
    min_df = trigger_df.copy()
    min_df['min_taken'] = min_taken
    min_df.dtypes
#     min_df.sort_values('min_taken', ascending=False)
    
#     trigger_df.loc[loss,:]
    #trigger_df.loc[profit,:]
    sym_data = {
        'symbol':symbol,
        'total_sells':len(trigger_df.loc[profit,:]),
        'total_loss':len(trigger_df.loc[loss,:]),
        'min_to_sell':round(min_df.min_taken.mean(),4),
        'profit': my_profit,
        'sold': trigger_df.sold.any().sum(),
        'lost': trigger_df.lost.any().sum(),
        'simple_buy': trigger_df.sma_buy
    }
#     print(trigger_df.head())

    sym_dict.append(sym_data)
    
    profit = trigger_df.loc[trigger_df['wol']=='profit','wol'].count()
    loss = trigger_df.loc[trigger_df['wol']=='loss','wol'].count()
    print(symbol,i)
    print(trigger_df.wol.value_counts())
    ind_roi = round((profit*(my_profit-1))-(loss*(1-my_loss)),5)
#     print('roi in 10 days: \n\n', ind_roi)
    ind_time = trigger_df.wol_time.mean()
#     print('Average trade time in minutes: ',ind_time)
    
    real_roi.append((symbol, ind_roi, ind_time, trigger_df.iloc[0,4]))

trades = pd.DataFrame.from_dict(sym_dict, orient='columns')
tradables = list()
tickers = list()
price = list()

# print(real_roi)
for x in real_roi:
    if x[2] < 12:
        tradables.append(x[1])
        tickers.append(x[0])
        price.append(x[3])

print( sum(tradables) )
print(tradables)
print(tickers)

FTI 0
profit    109
loss       99
Name: wol, dtype: int64
NYCB 1
loss      90
profit    60
Name: wol, dtype: int64
CDE 2
loss      248
profit     55
Name: wol, dtype: int64
BPFH 3
loss      20
profit    17
Name: wol, dtype: int64
MAC 4
loss      147
profit    104
Name: wol, dtype: int64
PBR.A 5
profit    96
loss      62
Name: wol, dtype: int64
QRTEA 6
profit    113
loss      105
Name: wol, dtype: int64
FCEL 7
loss      577
profit    420
Name: wol, dtype: int64
M 8
loss      332
profit    253
Name: wol, dtype: int64
PBR 9
loss      121
profit    119
Name: wol, dtype: int64
STKL 10
loss      52
profit    37
Name: wol, dtype: int64
CNX 11
loss      118
profit     88
Name: wol, dtype: int64
PVG 12
loss      36
profit    35
Name: wol, dtype: int64
ERIC 13
loss      115
profit     74
Name: wol, dtype: int64
UNIT 14
loss      118
profit     41
Name: wol, dtype: int64
ANGI 15
loss      121
profit     78
Name: wol, dtype: int64
CNHI 16
loss      42
profit    30
Name: wol, dtype: int64
HBAN 17
l

KeyError: 'candles'

In [7]:
tradables = list()
tickers = list()
price = list()
for x in real_roi:
    if x[2] < 20 and x[3] > 2:
        tradables.append(x[1])
        tickers.append(x[0])
        price.append(x[3])

print( sum(tradables) )
print(tickers)


0
[]


In [473]:
profit = trigger_df.wol.value_counts().profit
loss = trigger_df.wol.value_counts().loss
print(trigger_df.wol.value_counts())
print('roi in 10 days: \n\n', round((profit*(my_profit-1))-(loss*(1-my_loss)),5))
print('Average trade time in minutes: ',trigger_df.wol_time.mean())

profit    75
loss      32
Name: wol, dtype: int64
roi in 10 days: 

 0.088
Average trade time in minutes:  600.5140186915888


In [140]:
# data with out SMA
sold = trigger_df['sold'] == True
buy = trigger_df['bought'] == True
solo_sold = trigger_df.loc[sold & buy, 'bought'].count()  # total sold with out sma
solo_sold

sold = trigger_df['sold'] == False
buy = trigger_df['bought'] == True
solo_lost = trigger_df.loc[sold & buy, 'bought'].count() # total lost with out sma
solo_lost


# data with SMA
sold = trigger_df['sold'] == True
buy = trigger_df['bought'] == True
sma_true = trigger_df['sma_buy'] == True   # total bought with sma
sma_sold = trigger_df.loc[sold & buy & sma_true, 'bought'].count()
sma_sold

sold = trigger_df['sold'] == False
buy = trigger_df['bought'] == True
sma_true = trigger_df['sma_buy'] == True   # total lost with sma
sma_lost = trigger_df.loc[sold & buy & sma_true, 'bought'].count()
sma_lost


print('solo wins: ', solo_sold-solo_lost, '\nsma wins: ', sma_sold-sma_lost)

solo wins:  123 
sma wins:  47


In [25]:
trades.sort_values('min_to_sell', ascending=True).head(29)

,symbol,total_sells,total_loss,min_to_sell,profit,sold,lost
43,KATX,1,0,1.0000,1.0019,1,0
3,RMRK,0,0,1.0000,1.0019,1,0
39,ICLD,0,0,1.0000,1.0019,1,0
91,LBSR,0,0,1.0000,1.0019,1,0
28,SSOK,1,0,1.0000,1.0019,1,0
23,GAHC,1,0,1.0000,1.0019,1,0
12,RYCEY,14,3,1.0476,1.0019,1,0
49,AQMS,103,5,1.2129,1.0019,1,0
47,PMPG,1,0,2.0000,1.0019,1,0
27,PMCB,0,0,2.0000,1.0019,1,0


In [26]:
trades.sort_values('total_loss', ascending=True).sum()

symbol         IFXYGAHCDSCRRGBPPMCBSSOKBMXCINQDPFMSICLDNWGCKA...
total_sells                                                 7937
total_loss                                                  2404
min_to_sell                                          1.75286e+07
profit                                                    100.19
sold                                                          82
lost                                                           0
dtype: object

In [27]:
trades.loc[trades['min_to_sell']<6,:].symbol.values

array(['RMRK', 'OEG', 'RYCEY', 'XSPA', 'GAHC', 'PMCB', 'SSOK', 'LYG',
       'ICLD', 'KATX', 'PMPG', 'AQMS', 'TRCH', 'BKYI', 'ASRT', 'CBAT',
       'LBSR', 'DLOC'], dtype=object)

In [28]:
trades.loc[(trades['total_loss']==0)&(trades['min_to_sell']<10),:].symbol.values

array(['EVUS', 'RMRK', 'MJTV', 'GAHC', 'PMCB', 'SSOK', 'ICLD', 'KATX',
       'PMPG', 'LBSR', 'DLOC'], dtype=object)

In [29]:
trades.loc[(trades['total_loss']==0)&(trades['min_to_sell']<12),:].sum()

symbol         EVUSRMRKMJTVGAHCPMCBSSOKICLDKATXPMPGLBSRDLOC
total_sells                                              13
total_loss                                                0
min_to_sell                                         27.6286
profit                                              11.0209
sold                                                     11
lost                                                      0
dtype: object

In [21]:
trades.loc[(trades['total_loss']==0),:].sum()

symbol         VRUSIRNCVISMBLSPDRNGCOWIEFIRDPLSVSYMBIELGHHCPU...
total_sells                                                   14
total_loss                                                     0
min_to_sell                                              1593.95
profit                                                   33.0627
sold                                                          10
lost                                                           0
dtype: object

In [22]:
trades.loc[(trades['total_loss']==0),:].count()

symbol         33
total_sells    33
total_loss     33
min_to_sell    11
profit         33
sold           33
lost           33
dtype: int64

In [12]:
trades.loc[:,'profit'].iloc[0]

1.0019